# Generación base de datos
Este notebook trata de generar una base de datos a partir de unos archivos json que pueden encontrar en mi perfil de [GitHub](https://github.com/Nieterulz/nfr-extraction/tree/main/json).

Es importante que para que el código funcione se indique en la variable RUTA_JSON el directorio exacto donde se encuentran estos archivos.

### Importamos librerías necesarias

In [ ]:
import json
import pandas as pd
import os

### Definimos una función que devolverá el contenido que necesitamos del archivo JSON que le pasemos

In [ ]:
def readJson(file):
    jsonFile = open(RUTA_JSON + file, "r", encoding="utf8")
    jsonData = jsonFile.read()
    jsonData = json.loads(jsonData)
    jsonData = jsonData['content']
    return jsonData

### Definimos una función que devolverá un diccionario con los valores de clasificación para cada categoría

In [ ]:
def readClassifications(booleanClassifications):
    classifications = {}

    for c in categories:
        try:
            classifications[c] = int(
                booleanClassifications[c]["value"] == "TRUE")
            pass
        except Exception:
            classifications[c] = 0
            pass

    classifications["not applicable"] = 1 if(sum(classifications.values()) == 0) else 0

    return classifications

### Definimos una función que devolverá un dataframe con los datos que necesitamos del archivo json que pasemos como parámetro 

In [ ]:
def generateDataframe(file):
    jsonData = readJson(file)
    file = file.replace('.json', '')
    columns = ['sentences'] + categories
    db = pd.DataFrame(columns=columns)

    for element in jsonData:
        sentence = element['sentence']
        classification = readClassifications(element['booleanClassifications'])
        row = {
            'sentences': sentence,
        }
        for c in categories:
            row[c] = classification[c]

        db = db.append(row, ignore_index=True)

    return db

### El siguiente código se encarga de generar el archivo excel que será nuestra base de datos

In [ ]:
RUTA_JSON = "./drive/My Drive/json/" # Ruta donde estarán los json
RUTA_DB = "./drive/My Drive/xlsx/"

files = os.listdir(RUTA_JSON)


categories = [
    "access control", "audit", "availability", "legal", "look and feel",
    "maintainability", "operational", "privacy", "recoverability", "capacity and performance",
    "reliability", "security", "usability", "other nonfunctional", "functional", "not applicable"
]

columns = ['sentences'] + categories

db = pd.DataFrame(columns=columns)

files = files[1:]
for file in files:
    
        db = db.append(generateDataframe(file), ignore_index=True)

db.to_excel(RUTA_DB + "db.xlsx")